# HDF5 Dataset Builder

Use this notebook only to create HDF5 datasets from raw audio. Training, evaluation, and inference remain in the main notebook.


In [1]:
# Imports and configuration

import os
import json
import yaml
import time
import math
import random
import h5py
import librosa
import numpy as np
from pathlib import Path
from collections import defaultdict

# Paths (edit these)
BASE_DIR = Path(__file__).parent if '__file__' in globals() else Path.cwd()
# Notebook is inside IML-PW, so use local folders
data_root = BASE_DIR / "Recordings"
output_dir = BASE_DIR / "outputs"
output_dir.mkdir(parents=True, exist_ok=True)

if not data_root.exists():
    alt = BASE_DIR.parent / "IML-PW" / "Recordings"
    if alt.exists():
        print(f"⚠️ Using fallback data_root: {alt}")
        data_root = alt
    else:
        raise FileNotFoundError(f"Recordings folder not found. Checked: {data_root} and {alt}")

print(f"📁 data_root: {data_root}")
print(f"📁 output_dir: {output_dir}")

# Audio / feature params
sr = 16000
n_mels = 64
n_fft = 2048
hop_length = 512
chunk_duration = 3.0  # seconds

# Augmentation toggles
WITH_NOISE = True
WITH_SPEED = True
WITH_VTLP = False
WITH_SPECTRAL = True

random_seed = 42

SUPPORTED_EXTS = (".wav", ".mp3", ".m4a", ".wma")
np.random.seed(random_seed)
random.seed(random_seed)


📁 data_root: c:\Users\pczec\Desktop\Studia\SEM5\IML\IML-PW\Recordings
📁 output_dir: c:\Users\pczec\Desktop\Studia\SEM5\IML\IML-PW\outputs


In [2]:
# ==========================================================
# INFERENCE-COMPATIBLE PREPROCESSING CLASS
# ==========================================================
from scipy.signal import butter, lfilter
from pydub import AudioSegment


class AudioPreprocessor:
    """Lightweight preprocessing pipeline."""
    def __init__(self, sr=16000, n_mels=64, n_fft=2048, hop_length=512,
                 chunk_duration=3.0, remove_silence=True, normalize=True,
                 lowcut=None, highcut=None, filter_order=4):
        self.sr = sr
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.chunk_duration = chunk_duration
        self.remove_silence = remove_silence
        self.normalize = normalize
        self.lowcut = lowcut
        self.highcut = highcut
        self.filter_order = filter_order

    def load_audio(self, path):
        """Load and resample audio."""
        audio = AudioSegment.from_file(path)
        if audio.channels > 1:
            audio = audio.set_channels(1)

        samples = np.array(audio.get_array_of_samples()).astype(np.float32)
        samples /= (1 << (8 * audio.sample_width - 1))

        if audio.frame_rate != self.sr:
            samples = librosa.resample(samples, orig_sr=audio.frame_rate, target_sr=self.sr)

        return samples

    def trim_silence(self, samples, top_db=30):
        """Remove silence from audio."""
        non_silent, _ = librosa.effects.trim(samples, top_db=top_db)
        return non_silent

    def normalize_volume(self, samples):
        """Normalize audio volume."""
        max_val = np.max(np.abs(samples)) + 1e-9
        return samples / max_val

    def apply_filter(self, samples):
        """Apply Butterworth filter."""
        if not self.lowcut and not self.highcut:
            return samples

        nyq = 0.5 * self.sr
        if self.lowcut and self.highcut:
            b, a = butter(self.filter_order, [self.lowcut / nyq, self.highcut / nyq], btype="band")
        elif self.lowcut:
            b, a = butter(self.filter_order, self.lowcut / nyq, btype="high")
        else:
            b, a = butter(self.filter_order, self.highcut / nyq, btype="low")

        return lfilter(b, a, samples)

    def chunk_audio(self, samples):
        """Split audio into fixed-length chunks."""
        chunk_len = int(self.chunk_duration * self.sr)
        chunks = []

        for i in range(0, len(samples), chunk_len):
            chunk = samples[i:i + chunk_len]
            if len(chunk) < chunk_len:
                pad = np.zeros(chunk_len, dtype=samples.dtype)
                pad[:len(chunk)] = chunk
                chunk = pad
            chunks.append(chunk)

        return chunks

    def to_logmel(self, samples):
        """Convert audio to log-mel spectrogram."""
        mel = librosa.feature.melspectrogram(
            y=samples, sr=self.sr, n_mels=self.n_mels,
            n_fft=self.n_fft, hop_length=self.hop_length
        )
        logmel = librosa.power_to_db(mel, ref=np.max)
        return logmel.astype(np.float32)

    def get_config(self):
        """Return configuration dict."""
        return {
            "sr": self.sr,
            "n_mels": self.n_mels,
            "n_fft": self.n_fft,
            "hop_length": self.hop_length,
            "chunk_duration": self.chunk_duration,
            "remove_silence": self.remove_silence,
            "normalize": self.normalize,
            "lowcut": self.lowcut,
            "highcut": self.highcut,
            "filter_order": self.filter_order
        }


# ==========================================================
# AUGMENTATION FUNCTIONS
# ==========================================================
def add_ambient_noise(samples, noise_factor=0.005):
    """Add Gaussian noise."""
    noise = np.random.randn(len(samples)) * noise_factor
    return samples + noise


def speed_perturbation(samples, sr, speed_factor=1.0):
    """Apply time stretching."""
    return librosa.effects.time_stretch(samples, rate=speed_factor)


def spectral_augmentation(logmel, freq_mask_param=10, time_mask_param=20, n_masks=2):
    """Apply SpecAugment-style masking."""
    augmented = logmel.copy()
    n_mels, n_frames = augmented.shape

    for _ in range(n_masks):
        f = np.random.randint(0, min(freq_mask_param, n_mels))
        f0 = np.random.randint(0, max(1, n_mels - f))
        augmented[f0:f0+f, :] = augmented.mean()

    for _ in range(n_masks):
        t = np.random.randint(0, min(time_mask_param, n_frames))
        t0 = np.random.randint(0, max(1, n_frames - t))
        augmented[:, t0:t0+t] = augmented.mean()

    return augmented


def vocal_tract_length_perturbation(samples, sr, alpha=1.0):
    """Apply VTLP via frequency warping."""
    D = librosa.stft(samples)
    n_fft = (D.shape[0] - 1) * 2
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
    warped_freqs = freqs * alpha

    D_warped = np.zeros_like(D)
    for i, wf in enumerate(warped_freqs):
        if wf < freqs[-1]:
            idx = np.searchsorted(freqs, wf)
            if idx < len(freqs) - 1:
                D_warped[i] = D[idx]

    return librosa.istft(D_warped, length=len(samples))

In [3]:
# Helpers: YAML parsing and speaker mapping

def parse_labels_yaml(yaml_path):
    if not yaml_path.exists():
        return {}
    with open(yaml_path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f) or {}
    parsed = {}
    for filename, value in data.items():
        if isinstance(value, (list, tuple)) and len(value) >= 3:
            gender, in_group, speaker_name = value[0], value[1], value[2]
            parsed[filename] = {"gender": gender, "in_group": in_group, "speaker_name": speaker_name}
        elif isinstance(value, dict):
            parsed[filename] = {
                "gender": value.get("gender", "Unknown"),
                "in_group": value.get("in_group", False),
                "speaker_name": value.get("speaker_name", yaml_path.parent.name),
            }
    return parsed


def build_speaker_maps(root: Path):
    speaker_to_label = {}
    label_to_speaker = {}
    label_counter = 0
    # Collect from yaml
    for folder in root.iterdir():
        if not folder.is_dir():
            continue
        meta = parse_labels_yaml(folder / "labels.yaml")
        for info in meta.values():
            spk = info["speaker_name"]
            if spk not in speaker_to_label:
                speaker_to_label[spk] = label_counter
                label_to_speaker[label_counter] = spk
                label_counter += 1
    # Fallback to folder names
    for folder in root.iterdir():
        if folder.is_dir() and folder.name not in speaker_to_label:
            speaker_to_label[folder.name] = label_counter
            label_to_speaker[label_counter] = folder.name
            label_counter += 1
    return speaker_to_label, label_to_speaker


In [4]:
# Audio processing and augmentations

def load_audio(path: Path):
    """Robust audio loader: try soundfile, then librosa (warning-suppressed)."""
    # 1) Try soundfile (fast for WAV/FLAC/OGG)
    try:
        import soundfile as sf
        samples, file_sr = sf.read(str(path), always_2d=False)
        if samples.ndim > 1:
            samples = samples.mean(axis=1)
        samples = samples.astype(np.float32, copy=False)
        return samples, int(file_sr)
    except Exception:
        pass

    # 2) Fallback to librosa with audioread backend; suppress future warning
    import warnings as _warnings
    with _warnings.catch_warnings():
        _warnings.filterwarnings("ignore", category=FutureWarning, module="librosa.core.audio")
        samples, file_sr = librosa.load(str(path), sr=None, mono=True)
    samples = samples.astype(np.float32, copy=False)
    return samples, int(file_sr)


def resample_if_needed(samples, file_sr):
    if file_sr == sr:
        return samples
    return librosa.resample(samples, orig_sr=file_sr, target_sr=sr)


def chunk_audio(samples):
    chunk_len = int(chunk_duration * sr)
    chunks = []
    for i in range(0, len(samples), chunk_len):
        chunk = samples[i : i + chunk_len]
        if len(chunk) < chunk_len:
            pad = np.zeros(chunk_len, dtype=chunk.dtype)
            pad[: len(chunk)] = chunk
            chunk = pad
        chunks.append(chunk)
    return chunks


def to_logmel(chunk):
    mel = librosa.feature.melspectrogram(y=chunk, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    return librosa.power_to_db(mel, ref=np.max).astype(np.float32)


def add_noise(samples):
    noise_factor = np.random.uniform(0.002, 0.008)
    return samples + np.random.randn(len(samples)) * noise_factor, f"noise_{noise_factor:.4f}"


def speed_perturb(samples, factor):
    return librosa.effects.time_stretch(samples, rate=factor), f"speed_{factor:.2f}"


def vtlp_stub(samples, alpha):
    # Simplified placeholder; keeps timing but tags VTLP for label mapping
    return samples, f"vtlp_{alpha:.2f}"


def spectral_mask(logmel):
    lm = logmel.copy()
    n_m, n_t = lm.shape
    for _ in range(2):
        f = np.random.randint(0, min(10, n_m))
        f0 = np.random.randint(0, max(1, n_m - f))
        lm[f0 : f0 + f, :] = lm.mean()
    for _ in range(2):
        t = np.random.randint(0, min(20, n_t))
        t0 = np.random.randint(0, max(1, n_t - t))
        lm[:, t0 : t0 + t] = lm.mean()
    return lm

In [5]:
# Dataset traversal and split

def process_file(path: Path, speaker_label, speaker_info):
    samples, file_sr = load_audio(path)
    samples = resample_if_needed(samples, file_sr)
    results = []

    def logmels_from_samples(arr, tag, is_vtlp=False):
        for chunk in chunk_audio(arr):
            lm = to_logmel(chunk)
            if WITH_SPECTRAL:
                lm = spectral_mask(lm)
            results.append((lm, tag, is_vtlp))

    # original
    logmels_from_samples(samples, "original", False)

    if WITH_NOISE:
        noisy, tag = add_noise(samples)
        logmels_from_samples(noisy, tag, False)
    if WITH_SPEED:
        for f in (0.9, 1.1):
            try:
                sped, tag = speed_perturb(samples, f)
                logmels_from_samples(sped, tag, False)
            except Exception:
                pass
    if WITH_VTLP:
        for alpha in (0.9, 1.1):
            vtlp, tag = vtlp_stub(samples, alpha)
            logmels_from_samples(vtlp, tag, True)

    labels = []
    for _, tag, is_vtlp in results:
        if is_vtlp:
            labels.append(None)  # to be filled with VTLP mapping later
        else:
            labels.append(speaker_label)
    return results, labels, [speaker_info] * len(results)


def collect_data(root: Path):
    speaker_to_label, label_to_speaker = build_speaker_maps(root)
    specs, labels, aug_tags, is_vtlp_flags, spk_infos, file_paths = [], [], [], [], [], []
    vtlp_map = {}
    next_vtlp = 10000

    for folder in sorted(root.iterdir()):
        if not folder.is_dir():
            continue
        meta = parse_labels_yaml(folder / "labels.yaml")
        for audio_file in sorted(folder.iterdir()):
            if not audio_file.suffix.lower() in SUPPORTED_EXTS:
                continue
            info = meta.get(audio_file.name, {"speaker_name": folder.name, "gender": "Unknown", "in_group": True})
            spk = info["speaker_name"]
            speaker_label = speaker_to_label[spk]
            res, lbls, infos = process_file(audio_file, speaker_label, info)
            for (lm, tag, is_vtlp), lbl, inf in zip(res, lbls, infos):
                if is_vtlp:
                    key = (speaker_label, tag)
                    if key not in vtlp_map:
                        vtlp_map[key] = next_vtlp
                        next_vtlp += 1
                    lbl = vtlp_map[key]
                specs.append(lm)
                labels.append(lbl)
                aug_tags.append(tag)
                is_vtlp_flags.append(is_vtlp)
                spk_infos.append(inf)
                file_paths.append(str(audio_file))
    return specs, labels, aug_tags, is_vtlp_flags, spk_infos, file_paths, speaker_to_label, label_to_speaker, vtlp_map


def split_80_10_10(file_paths, specs, labels, aug_tags, is_vtlp_flags, spk_infos):
    file_to_idx = defaultdict(list)
    for i, fp in enumerate(file_paths):
        file_to_idx[fp].append(i)
    files = list(file_to_idx.keys())
    random.shuffle(files)
    n = len(files)
    n_train = int(0.8 * n)
    n_test = int(0.1 * n)
    train_files = set(files[:n_train])
    test_files = set(files[n_train : n_train + n_test])
    val_files = set(files[n_train + n_test :])

    def gather(selected):
        idxs = [i for f in selected for i in file_to_idx[f]]
        return {
            "specs": [specs[i] for i in idxs],
            "labels": [labels[i] for i in idxs],
            "aug_tags": [aug_tags[i] for i in idxs],
            "is_vtlp": [is_vtlp_flags[i] for i in idxs],
            "file_paths": [file_paths[i] for i in idxs],
            "speaker_info": [spk_infos[i] for i in idxs],
        }

    splits = {
        "train": gather(train_files),
        "val": gather(val_files),
        "test": gather(test_files),
    }
    return splits


In [6]:
# HDF5 writing

def save_h5(splits, speaker_to_label, label_to_speaker, vtlp_map, out_path: Path):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with h5py.File(out_path, "w") as f:
        for split_name, data in splits.items():
            if len(data["specs"]) == 0:
                continue
            n_items = len(data["specs"])
            n_mels_local = data["specs"][0].shape[0]
            max_T = max(s.shape[1] for s in data["specs"])
            arr = np.full((n_items, n_mels_local, max_T), fill_value=-80.0, dtype=np.float32)
            lengths = np.zeros(n_items, dtype=np.int64)
            for i, s in enumerate(data["specs"]):
                T = s.shape[1]
                arr[i, :, :T] = s
                lengths[i] = T
            grp = f.create_group(split_name)
            grp.create_dataset("logmel", data=arr, compression="gzip")
            grp.create_dataset("label", data=np.array(data["labels"], dtype=np.int64), compression="gzip")
            grp.create_dataset("length", data=lengths, compression="gzip")
            dt = h5py.string_dtype(encoding="utf-8")
            grp.create_dataset("augmentation", data=np.array(data["aug_tags"], dtype=dt), compression="gzip")
            grp.create_dataset("file_path", data=np.array(data["file_paths"], dtype=dt), compression="gzip")
        meta = f.create_group("meta")
        meta.attrs.update({
            "sample_rate": sr,
            "feature_type": "log-mel spectrogram",
            "total_speakers": len(speaker_to_label),
        })
        file_desc = {
            "dataset_name": out_path.name,
            "preprocessing_config": {
                "sr": sr,
                "n_mels": n_mels,
                "n_fft": n_fft,
                "hop_length": hop_length,
                "chunk_duration": chunk_duration,
                "with_noise": WITH_NOISE,
                "with_speed": WITH_SPEED,
                "with_vtlp": WITH_VTLP,
                "with_spectral": WITH_SPECTRAL,
            },
            "split_strategy": "80-10-10 by file",
        }
        meta.create_dataset("file_description.yaml", data=np.bytes_(yaml.safe_dump(file_desc)))
        speaker_mapping = {k: int(v) for k, v in speaker_to_label.items()}
        meta.create_dataset("speaker_mapping.yaml", data=np.bytes_(yaml.safe_dump({"speakers": speaker_mapping})))
        meta.create_dataset("split_statistics.json", data=np.bytes_(json.dumps({}, indent=2).encode("utf-8")))
    print(f"Saved HDF5 → {out_path}")


In [7]:
# Run: build and save HDF5

start = time.time()
print("Collecting data...")
(specs, labels, aug_tags, is_vtlp_flags, spk_infos, file_paths,
 speaker_to_label, label_to_speaker, vtlp_map) = collect_data(data_root)
print(f"Total chunks: {len(specs)} | Base speakers: {len(speaker_to_label)} | VTLP variants: {len(vtlp_map)}")

print("Building splits (80/10/10 by file)...")
splits = split_80_10_10(file_paths, specs, labels, aug_tags, is_vtlp_flags, spk_infos)

# Include date and time to avoid collisions
out_name = f"logmels_{'aug' if any([WITH_NOISE, WITH_SPEED, WITH_VTLP, WITH_SPECTRAL]) else 'noaug'}_{time.strftime('%y-%m-%d_%H-%M-%S')}.h5"
out_path = output_dir / out_name

print("Saving HDF5...")
save_h5(splits, speaker_to_label, label_to_speaker, vtlp_map, out_path)
print(f"Done in {time.time() - start:.1f}s")
print(f"Saved to: {out_path}")

C:\Users\pczec\AppData\Local\Temp\ipykernel_32212\415391012.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, file_sr = librosa.load(str(path), sr=None, mono=True)
C:\Users\pczec\AppData\Local\Temp\ipykernel_32212\415391012.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, file_sr = librosa.load(str(path), sr=None, mono=True)
C:\Users\pczec\AppData\Local\Temp\ipykernel_32212\415391012.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, file_sr = librosa.load(str(path), sr=None, mono=True)
C:\Users\pczec\AppData\Local\Temp\ipykernel_32212\415391012.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, file_sr = librosa.load(str(path), sr=None, mono=True)
C:\Users\pczec\AppData\Local\Temp\ipykernel_32212\415391012.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, file_sr = librosa.load(str(path), sr=None, mono=True)
C:\Users\pczec\AppData\Local\Temp\ipykernel_3

Total chunks: 37061 | Base speakers: 5 | VTLP variants: 0
Building splits (80/10/10 by file)...
Saving HDF5...
Saved HDF5 → c:\Users\pczec\Desktop\Studia\SEM5\IML\IML-PW\outputs\logmels_aug_26-01-25_10-04-39.h5
Done in 507.2s
Saved to: c:\Users\pczec\Desktop\Studia\SEM5\IML\IML-PW\outputs\logmels_aug_26-01-25_10-04-39.h5
